In [1]:
import os
import gc
import requests
from bs4 import BeautifulSoup
import pandas as pd
import zipfile
import re
import time

In [2]:
from etl import *

In [3]:
ld_to_csv('data/unzipped')

creat 5-million-row table 1
creat 5-million-row table 2
creat 5-million-row table 3
creat 5-million-row table 4
creat 5-million-row table 5
creat 5-million-row table 6
creat 5-million-row table 7
creat 5-million-row table 8
creat 5-million-row table 9
creat 5-million-row table 10
creat 5-million-row table 11
creat 5-million-row table 12
creat 5-million-row table 13
creat 5-million-row table 14
creat 5-million-row table 15
creat 5-million-row table 16
creat 5-million-row table 17
creat 5-million-row table 18
creat 5-million-row table 19
creat 5-million-row table 20
creat 5-million-row table 21
creat 5-million-row table 22
creat 5-million-row table 23
creat 5-million-row table 24
creat 5-million-row table 25
creat 5-million-row table 26
creat 5-million-row table 27
creat 5-million-row table 28
creat 5-million-row table 29
creat 5-million-row table 30
creat 5-million-row table 31
creat 5-million-row table 32
creat 5-million-row table 33
creat 5-million-row table 34
creat 5-million-row tab

In [3]:
def add_dicts(dict1, dict2):
    result={}
    if len(dict1.keys())==0:
        return dict2
    elif len(dict2.keys())==0:
        return dict1
    else:
        for key in dict1.keys():
            if key in dict2.keys():
                result[key]=dict1[key]+dict2[key]
            else:
                result[key]=dict1[key]
        for key2 in dict2.keys():
            if key2 not in result.keys():
                result[key2]=dict2[key2]
        return result
                

In [4]:
def get_csvs_route(road):
    result=[]
    for filename in os.listdir(road):
        fileroute=os.path.join(road,filename)
        result.append(fileroute)
    return result

In [5]:
def bots_raw_dict(road):
    total_freq={}
    revert_freq={}
    for csvname in os.listdir(road):
        csvroute=os.path.join(road, csvname)
        csvdict={}
        df=pd.read_csv(csvroute)
        df['user']=df['user'].str.lower()
        df['revert']=df['revert'].astype('int')
        dfrevert=df[df['revert']==1]
        bots=df[df['user'].str.contains('bot')]
        reverts=bots[bots['revert']==1]
        thef=bots['user'].value_counts().to_dict()
        ther=reverts['user'].value_counts().to_dict()
        total_freq=add_dicts(total_freq, thef)
        revert_freq=add_dicts(revert_freq, ther)
        print('finish dict csv', csvname, len(total_freq))
        del thef, ther
        del [[df, bots, reverts, dfrevert]]
        gc.collect()
        df=pd.DataFrame()
        bots=pd.DataFrame()
        reverts=pd.DataFrame()
        thef={}
        ther={}
        dfrevert=pd.DataFrame()
    return total_freq, revert_freq

In [5]:
total_freq, revert_freq=bots_raw_dict('data/csvs/en_wiki')

finish dict csv en_wiki_13.csv 781
finish dict csv en_wiki_14.csv 1037
finish dict csv en_wiki_21.csv 1230
finish dict csv en_wiki_26.csv 1425
finish dict csv en_wiki_28.csv 1608
finish dict csv en_wiki_1.csv 1761
finish dict csv en_wiki_42.csv 1946
finish dict csv en_wiki_39.csv 2090
finish dict csv en_wiki_6.csv 2212
finish dict csv en_wiki_37.csv 2311
finish dict csv en_wiki_8.csv 2432
finish dict csv en_wiki_30.csv 2548
finish dict csv en_wiki_29.csv 2645
finish dict csv en_wiki_27.csv 2713
finish dict csv en_wiki_20.csv 2811
finish dict csv en_wiki_15.csv 2912
finish dict csv en_wiki_12.csv 2997
finish dict csv en_wiki_31.csv 3091
finish dict csv en_wiki_36.csv 3174
finish dict csv en_wiki_9.csv 3251
finish dict csv en_wiki_38.csv 3361
finish dict csv en_wiki_7.csv 3429
finish dict csv en_wiki_44.csv 3504
finish dict csv en_wiki_43.csv 3598
finish dict csv en_wiki_35.csv 3652
finish dict csv en_wiki_32.csv 3710
finish dict csv en_wiki_3.csv 3773
finish dict csv en_wiki_40.csv 3839

In [6]:
total_f=pd.Series(total_freq)

In [7]:
total_f.describe()

count    4.873000e+03
mean     3.389500e+03
std      4.717530e+04
min      1.000000e+00
25%      1.000000e+00
50%      3.000000e+00
75%      1.100000e+01
max      2.695952e+06
dtype: float64

In [14]:
total_f.sort_values(ascending=False).head(20)

smackbot                   2695952
cydebot                     763366
cluebot                     644069
lightbot                    638751
russbot                     606590
the_anomebot2               536912
thijs!bot                   418576
alaibot                     379167
full-date_unlinking_bot     322808
bluebot                     320537
siebot                      291204
botijo                      289988
yurikbot                    272996
xqbot                       261936
cmdrobot                    234560
erik9bot                    233001
txikibot                    223028
flabot                      212084
dumzibot                    208023
volkovbot                   207434
dtype: int64

In [12]:
total_r=pd.Series(revert_freq)

In [13]:
total_r.sort_values(ascending=False).head(20)

cluebot                643853
antivandalbot          141364
voabot_ii              134472
xlinkbot               103205
martinbot              100789
tawkerbot2              69941
pseudobot               48919
russbot                 25174
xqbot                   19643
antispambot             19338
darknessbot             13990
tawkerbot4              13533
soxbot_iii              12911
cydebot                  9175
dashbot                  8093
countervandalismbot      7379
タチコマ_robot               5775
dumbbot                  4776
scepbot                  4301
robotman1974             2733
dtype: int64

In [6]:
def calculateM(df):
    # find revert pairs
    revert_pairs = []
    revert_users = []
    ones=df[df['revert']==1]
    twodf=df[df['revert']==0]
    for oness in ones['user'].unique():
        masker=ones[ones['user']==oness]
        for indi in masker['version'].values:
            one = oness
            the_version=indi+1
            alti=the_version-1
            try:
                twocolumn=twodf[twodf['version']==the_version]
                two=twocolumn['user'].values[0]
            except Exception as e:
                twocolumn=twodf[twodf['version']==alti]
                two=twocolumn['user'].values[0]
            revert_pairs.append((one, two))
            if one not in revert_users:
                revert_users.append(one)
            if two not in revert_users:
                revert_users.append(two)
    #find mutual reverts
    mutual_rev_users = []
    for pair in revert_pairs:
        one = pair[0]
        two = pair[1]
    
        #mutual revert found
        if (two, one) in revert_pairs:
            mutual_rev_users.append(two)
            mutual_rev_users.append(one)

    #remove duplicates, calculate num
    E = len(list(set(mutual_rev_users)))
    #calculate M
    M = 0
    num_edits = df['user'].value_counts()
    revert_pairs = list(set(revert_pairs))
    for pair in revert_pairs:
        one = pair[0]
        two = pair[1]
        if num_edits[one] < num_edits[two]:
            N = num_edits[one]
        else:
            N = num_edits[two]

        M += N

    M *= E
    return M

In [7]:
def better_calculate_M(road):
    Ms={}
    for csvname in os.listdir(road):
        st=time.time()
        csvroute=os.path.join(road, csvname)
        csvdict={}
        df=pd.read_csv(csvroute)
        df['revert']=df['revert'].astype('int')
        df['version']=df['version'].astype('int')
        first_clear=df.groupby('article')['revert'].sum()
        fs=first_clear[first_clear<2].to_dict()
        for k in fs:
            fs[k]=0
        Ms.update(fs)
        need_calculate=first_clear[first_clear>1].index
        for i in need_calculate:
            partdf=df.loc[df['article']==i, :]
            M=calculateM(partdf)
            Ms[i]=M
            del [[partdf]]
            gc.collect()
            partdf=pd.DataFrame()
        totalt=time.time()-st
        print('complete M calculation on', csvname, 'at the time used in seconds', totalt)
        del [[df, first_clear, need_calculate]]
        gc.collect()
        df=pd.DataFrame()
        first_clear=pd.Series()
    return Ms
    

In [8]:
import warnings
warnings.filterwarnings('error')

In [9]:
Ms={}
df=pd.read_csv('data/csvs/en_wiki/en_wiki_1.csv')
first_clear=df.groupby('article')['revert'].sum()
fs=first_clear[first_clear<2].to_dict()
for k in fs:
    fs[k]=0
Ms.update(fs)
need_calculate=first_clear[first_clear>1].index
len(need_calculate)

26897

In [13]:
Ms=better_calculate_M('data/csvs/en_wiki')

KeyboardInterrupt: 